<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/Temporal_Gradient_Descent_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import tensorflow as tf

# Reproducibility
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

def make_temporal_data(n_samples=12000, length=40, n_classes=10,
                       noise_std=0.15, burst_prob=0.08, seed=SEED):
    """
    Build labelable 1D time series:
    - Class c has frequency fc and phase phic; optional trend and bursts.
    - Add AR(1) noise for temporal correlation.
    Output shape: (n_samples, length, 1), labels: (n_samples,)
    """
    rng = np.random.default_rng(seed)
    per_class = n_samples // n_classes
    X, y = [], []

    freqs = np.linspace(0.6, 2.0, n_classes)           # distinct frequencies
    phases = np.linspace(0.0, np.pi, n_classes)        # distinct phases
    t = np.linspace(0, 1, length)

    for c in range(n_classes):
        f, ph = freqs[c], phases[c]
        for _ in range(per_class):
            base = np.sin(2*np.pi*f*t + ph)
            trend = 0.4 * (t - 0.5) * ((c % 2)*2 - 1)   # alternating up/down trend
            signal = 0.8*base + 0.2*trend

            # AR(1) noise
            eps = rng.normal(0, noise_std, size=length)
            ar = np.zeros_like(eps)
            phi = 0.6
            for i in range(1, length):
                ar[i] = phi * ar[i-1] + eps[i]

            x = signal + ar

            # Occasional burst
            if rng.random() < burst_prob:
                k = rng.integers(5, length-5)
                x[k:k+3] += rng.normal(1.5, 0.3)

            X.append(x[:, None].astype("float32"))
            y.append(c)

    X = np.stack(X, axis=0)
    y = np.array(y, dtype=np.int32)

    # Shuffle
    idx = rng.permutation(len(X))
    return X[idx], y[idx]

# Data
X, y = make_temporal_data(n_samples=12000, length=40, n_classes=10, noise_std=0.12, burst_prob=0.06)

# Split 70/15/15
n = len(X)
n_train = int(0.70 * n); n_val = int(0.15 * n)
X_train, y_train = X[:n_train], y[:n_train]
X_val,   y_val   = X[n_train:n_train+n_val], y[n_train:n_train+n_val]
X_test,  y_test  = X[n_train+n_val:],       y[n_train+n_val:]

# Standardize by train stats (per feature/channel)
mean = X_train.mean(axis=(0,1), keepdims=True)
std  = X_train.std(axis=(0,1), keepdims=True) + 1e-7
X_train = (X_train - mean) / std
X_val   = (X_val   - mean) / std
X_test  = (X_test  - mean) / std

# tf.data pipelines
BATCH = 128
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(8192, seed=SEED).batch(BATCH).prefetch(tf.data.AUTOTUNE)
val_ds   = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(BATCH).prefetch(tf.data.AUTOTUNE)
test_ds  = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(BATCH).prefetch(tf.data.AUTOTUNE)

# Temporal Intelligence Model with learned retrocausality
class TemporalAI(tf.keras.Model):
    def __init__(self, n_classes=10, width=64, l2=1e-5, drop=0.2):
        super().__init__()
        self.past_layer = tf.keras.layers.Conv1D(width, kernel_size=5, padding='same', activation='relu',
                                                 kernel_regularizer=tf.keras.regularizers.l2(l2))
        self.present_layer = tf.keras.layers.GRU(2*width, return_sequences=False,
                                                 kernel_regularizer=tf.keras.regularizers.l2(l2))
        self.future_layer = tf.keras.layers.Dense(width, activation='relu',
                                                  kernel_regularizer=tf.keras.regularizers.l2(l2))
        self.proj_past = tf.keras.layers.Dense(width, activation=None)
        self.gate = tf.keras.layers.Dense(width, activation='sigmoid')
        self.dropout = tf.keras.layers.Dropout(drop)
        self.global_pool = tf.keras.layers.GlobalAveragePooling1D()
        self.output_layer = tf.keras.layers.Dense(n_classes, activation='softmax')

    def call(self, inputs, training=False):
        # Past: local temporal context via Conv1D + pooling
        past_feat = self.past_layer(inputs)
        past_ctx = self.global_pool(past_feat)          # (batch, width)
        past_ctx = self.proj_past(past_ctx)             # align dims

        # Present: sequence state via GRU
        present_state = self.present_layer(past_feat)   # (batch, 2*width)

        # Future: projected hypothesis of what's ahead
        future_feat = self.future_layer(present_state)  # (batch, width)

        # Learned retrocausal gate
        gate_in = tf.concat([past_ctx, future_feat], axis=-1)
        g = self.gate(gate_in)                          # (batch, width) in [0,1]
        time_optimized = (1.0 - g) * past_ctx + g * future_feat
        time_optimized = self.dropout(time_optimized, training=training)

        return self.output_layer(time_optimized)

model = TemporalAI(n_classes=10, width=64, l2=1e-5, drop=0.2)

# Optimizer and compile
try:
    optimizer = tf.keras.optimizers.AdamW(learning_rate=1e-3, weight_decay=1e-4)
except AttributeError:
    # Fallback if AdamW is unavailable
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Callbacks & logging
os.makedirs("results", exist_ok=True)
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=8, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, min_lr=1e-5, verbose=1),
    tf.keras.callbacks.CSVLogger("results/training_history.csv"),
    tf.keras.callbacks.ModelCheckpoint("results/temporal_best.keras", monitor="val_accuracy",
                                       save_best_only=True)
]

history = model.fit(train_ds, validation_data=val_ds, epochs=60, callbacks=callbacks, verbose=1)

# Evaluate
test_loss, test_acc = model.evaluate(test_ds, verbose=0)
print(f"✅ Test accuracy: {test_acc:.4f} | Test loss: {test_loss:.4f}")
print("✅ Training history saved to results/training_history.csv")
print("✅ Best model saved to results/temporal_best.keras")

# Confusion matrix
y_pred = model.predict(test_ds, verbose=0).argmax(axis=1)
cm = tf.math.confusion_matrix(y_test, y_pred, num_classes=10).numpy()
print("Confusion matrix:\n", cm)

# If you prefer to keep your original Dense-only architecture:
# X_flat = X.reshape(len(X), -1); use Dense layers on X_flat.